# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы . Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработку данных здесь делать не нужно.

Необходимо построить модель с максимально большим значением *accuracy*. Минимально допустимое значение *accuracy* 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
Нужно подобрать модель для задачи классификации, которая порекомендует пользователю один из предложенных тарифов. Исходные данные - описание поведения пользователей.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('C:/Users/hk/Desktop/практика/датасеты/users_behavior.csv')

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


**Описание данных**
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- `calls` - количество звонков;
- `minutes` - суммарная длительность звонков в минутах;
- `messages` - количество sms-сообщений;
- `mb_used` - израсходованный интернет-трафик в Мб;
- `is_ultra` - каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df['is_ultra'].value_counts()

is_ultra
0    2229
1     985
Name: count, dtype: int64

Приведем тип данных в колонках `calls` и `messages` к int и проверим данные на явные дубликаты

In [6]:
for column in df[['calls','messages']]:
    df[column] = df[column].astype('int')

In [7]:
df.duplicated().sum()

0

In [8]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Вывод по полученному датафрейму**

В исходном датафрейме 3214 строк, из них
- для пользователей с тарифом Смарт(0) - 2229 признаков,
- для пользователей с тарифом Ультра(1) - 985 признаков
В целом исходные "чистые" и могут быть использованы для формирования моделей (пропусков и дубликатов в датафрейме нет)

## Разбейте данные на выборки

Искомая наилучшая модель должна предсказать, на какой тариф лучше перейти пользователю, значит целевой признак в этой задаче - is_ultra. Целевой признак категориальный (два тарифа 0 и 1), поэтому будем решать задачу классификации.

Разобьем данные на обучающие признаки и целовой признак

In [9]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Так как в представленном датафрейме скрытой тестовой выборки нет, данные нужно разбить на три части: обучающую, валидационную и тестовую в соотношении 3:1:1 соответственно

Определим обучающую выборку

In [10]:
features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                           test_size = 0.4,
                                                                           random_state=12345)

Посмотрим на размеры обучающей выборки:

In [11]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


Определим тестовую и валидационную выборки

In [12]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test,
                                                                            target_test,
                                                                           test_size = 0.5,
                                                                           random_state=12345)

Посмотрим на размер валидационной и тестовой выборок

In [13]:
print(features_test.shape)
print(target_test.shape)
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)
(643, 4)
(643,)


**Вывод**

Получили разделенные на три части в соотношении 3:1:1 выборки:
- обучающая 60% (1928 признаков);
- тестовая 20% (643 признака);
- валидационная 20% (643 признака)

## Исследуйте модели

### Модель DecisionTreeClassifier (Дерево решений)

Обучим модель дерева решений на обучающей выборке features_train, target_train.
Переберем вложенным циклом гиперпараметр `max_depth`, и найдем модель с гиперпараметром, которая дает наилучший результат доли правильных ответов (accuracy) на валидационной выборке и сохраним результат в best_result_dtc

In [14]:
best_model_dtc = None
best_depth_dtc = 0
best_result_dtc = 0
best_samples_split_dtc = 0
best_samples_leaf_dtc = 0
for leafs in range(1, 5, 1):
    for depth in range(1, 6):
        for samples in range(10, 100, 10):
            model_dtc = DecisionTreeClassifier(random_state=12345,
                                               max_depth=depth,
                                               min_samples_split=samples,
                                               max_features='log2', 
                                               min_samples_leaf=leafs)
            model_dtc.fit(features_train, target_train)
            predictions_valid_dtc = model_dtc.predict(features_valid)
            result_dtc = accuracy_score(target_valid, predictions_valid_dtc)
            if result_dtc > best_result_dtc:
                best_model_dtc =  model_dtc #сохраняем лучшую модель
                best_depth_dtc = depth #сохраняем лучшую глубину дерева
                best_result_dtc = result_dtc #сохраняем лучшее значение accuracy на валидационной выборке
                best_samples_split_dtc = samples #сохраняем лучшее "минимальное" количество выборок 
                best_samples_leaf_dtc = leafs #сохраняем лучшее "Минимальное" количество выборок,
                                              #необходимое для нахождения в конечном узле

In [15]:
print('Accuracy лучшей модели дерева решений:', best_result_dtc)
print('Глубина дерева лучшей модели', best_depth_dtc)
print('Минимальное количество выборок', best_samples_split_dtc)
print('Минимальное количество выборок, необходимое для нахождения в конечном узле',  best_samples_leaf_dtc)

Accuracy лучшей модели дерева решений: 0.7947122861586314
Глубина дерева лучшей модели 5
Минимальное количество выборок 20
Минимальное количество выборок, необходимое для нахождения в конечном узле 3


**Вывод**

Наилучший результат предсказаний по валидационной выборке показала модель дерева решений с гиперпараметрами:
- random_state = 12345;
- max_depth = 5;
- min_samples_split = 20;
- min_samples_leaf = 3;
- max_features='log2'

Доля правильных ответов на валидационной выборке при таких значениях гиперпараметров составила 79,47%

### Модель RandomForestClassifier (Случайный лес)

Обучим модель случайного леса на обучающей выборке features_train, target_train.
Переберем вложенным циклом гиперпараметры `max_depth` и `n_estimators`, и найдем модель с гиперпараметрами, которая дает наилучший результат доли правильных ответов (accuracy) на валидационной выборке и сохраним результат в best_result_dtc

In [16]:
best_model_rfc = None
best_depth_rfc = 0
best_result_rfc = 0
best_samples_split_rfc = 0
best_est_rfc = 0
best_samples_leaf_rfc = 0
for leafs in range(1, 5, 1):
    for samples in range(10, 100, 10):
        for est in range(1, 20):
            for depth in range(1, 11):
                model_rfc = RandomForestClassifier(random_state=12345,
                                                   n_estimators=est,
                                                   max_depth=depth,
                                                   min_samples_split=samples,
                                                   max_features='log2',
                                                   min_samples_leaf=leafs)
                model_rfc.fit(features_train, target_train)
                predictions_valid_rfc = model_rfc.predict(features_valid)
                result_rfc = accuracy_score(target_valid, predictions_valid_rfc)
                if result_rfc > best_result_rfc:
                    best_model_rfc =  model_rfc #сохраняем лучшую модель
                    best_depth_rfc = depth #сохраняем лучшую глубину дерева
                    best_result_rfc = result_rfc #сохраняем лучшее значение accuracy на валидационной выборке
                    best_est_rfc = est #сохраняем лучшее значение количества дереьвев 
                    best_samples_split_rfc = samples #сохраняем лучшее "минимальное" количество выборок
                    best_samples_leaf_rfc = leafs #сохраняем лучшее "Минимальное" количество выборок, 
                                                  #необходимое для нахождения в конечном узле

In [17]:
print('Accuracy наилучшей модели на валидационной выборке:',  best_result_rfc)    
print('Количество деревьев в наилучшей модели', best_est_rfc)
print('Глубина деревьев', best_depth_rfc)
print('Минимальное количество выборок', best_samples_split_rfc)
print('Минимальное количество выборок, необходимое для нахождения в конечном узле',  best_samples_leaf_rfc)

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247
Количество деревьев в наилучшей модели 8
Глубина деревьев 6
Минимальное количество выборок 10
Минимальное количество выборок, необходимое для нахождения в конечном узле 4


**Вывод**

Наилучший результат предсказаний по валидационной выборке показала модель случайного леса с гиперпараметрами:
- random_state = 12345;
- max_depth = 6;
- n_estimators = 8;
- min_samples_split = 10;
- min_samples_leaf = 4;
- max_features='log2'

Доля правильных ответов на валидационной выборке с учетом таких гиперпараметров составила 80,87%

### Модель Logistic Regression (Логистическая регрессия)

Обучим модель логистической регрессиии с гиперпараметрами `random_state` и `solver='lbfgs'`

In [18]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs')
model_lr.fit(features_train, target_train)
predictions_valid_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(predictions_valid_lr, target_valid)
print('Accuracy Логистической регрессии:', accuracy_lr, end ='')

Accuracy Логистической регрессии: 0.7107309486780715

Доля правильных ответов составила меньше требуемого в 0,75, поэтому добавим гиперпараметр `penalty` в логистической регрессии

In [19]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', penalty=None)
model_lr.fit(features_train, target_train)
predictions_valid_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(predictions_valid_lr, target_valid)
print('Accuracy Логистической регрессии:', accuracy_lr, end ='')

Accuracy Логистической регрессии: 0.7558320373250389

**Вывод**

Таким образом, меняя и добавляя гиперпараметры в логистической регресии достигли минимально требуемой доли правильных ответов 75,58%

Создадим небольшую функцию, формирующую датафрейм с полученными точностями для трех видов рассматриваемых моделей

In [20]:
def df_accuracy(m1_tree, m2_forest, m3_reg):
    data = pd.DataFrame({'accuracy': [m1_tree, 
                                      m2_forest, 
                                      m3_reg]}, index=['Дерево решений', 
                                                       'Случайный лес', 
                                                       'Логистическая регрессия']).sort_values(by='accuracy', 
                                                                                               ascending=False)
    return data

In [21]:
df_accuracy(m1_tree=best_result_dtc, m2_forest=best_result_rfc, m3_reg=accuracy_lr)

,accuracy
Случайный лес,0.808709
Дерево решений,0.794712
Логистическая регрессия,0.755832


### Вывод по разделу

Таким образом получаем, что наибольшей точностью по валидационной выборке обладает модель случайного леса - 80,87%

## Проверьте модель на тестовой выборке

In [22]:
tree_test =  best_model_dtc.score(features_test, target_test) # дерево
forest_test =  best_model_rfc.score(features_test, target_test) # лес
log_reg_test = model_lr.score(features_test, target_test) # лог. регрессия

Выведем на экран время, затраченное каждой моделью на предсказание

In [23]:
%%time
predictions_test_dtc = model_dtc.predict(features_test)

CPU times: total: 0 ns
Wall time: 998 µs


In [24]:
%%time
predictions_test_rfc = model_rfc.predict(features_test)

CPU times: total: 0 ns
Wall time: 4.02 ms


In [25]:
%%time
predictions_test_lr = model_lr.predict(features_test)

CPU times: total: 0 ns
Wall time: 998 µs


Выведем на экран в порядке убывания значения точности accuracy для каждой модели на тестовой выборке.

In [26]:
df_accuracy(m1_tree=tree_test, m2_forest=forest_test, m3_reg=log_reg_test)

,accuracy
Случайный лес,0.802488
Дерево решений,0.785381
Логистическая регрессия,0.738725


**Вывод**

На тестовой выборке получили следующие результаты:
- наилучшие показатели точности у случайного леса (80,24%), но при этом самое долгое время предсказаний;
- неплохие показатели качества  у дерева решений (78,53%) и быстрое время предсказаний;
- наименьшеие показатели точности у логистической регрессии (73,87%), на при этом самое быстрое время предсказаний

## Проверка модели на адекватность

Создадим модель DummyClassifier, в которой все предсказания будут равны наиболее часто встречающемуся признаку в датасете - 0 - тарифу смарт.
Оценим долю правильных ответов модели и сравним с полученной точностью моделей дерева решений/случайного леса.

In [27]:
clf = DummyClassifier(strategy='most_frequent', random_state=12345)

clf.fit(features_train, target_train)

result_dummy = clf.score(features_test, target_test)

print('Точность оценки модели Dummy на тестовой выборке:', result_dummy)

Точность оценки модели Dummy на тестовой выборке: 0.6842923794712286


Таким образом получили, что доля правильных ответов моделей дерева решений и случайного леса примерно на 9-10% выше по сравнению с моделью DummyClassifier

## Общий вывод

Целью данного исследования было подобрать модель, рекомендующую пользователям наиболее подходящий для них тариф - смарт или ультра.
Для этого, в рамках решения задачи классификации было проведено обучение трех моделей:

- дерева решений - DecisionTreeClassifier,
- случайного леса - RandomForestClassifier,
- логистической регрессии - LogisticRegression.

При этом, исходный датасет был разделен на три выборки: обучающую, валидационную и тестовую (60%, 20%, 20% соответветственно).

При обучении на обучающей выборке и тестировании моделей на валидационной выборке:
- наилучший результат показала модель случайного леса с гиперпараметрами random_state=12345, n_estimators=12, max_depth=6 - точность предсказаний составила 80,40%.
- Второй по точности предсказаний оказалась модель дерева решений с гиперпараметрами random_state=12345, max_depth=3, min_samples_split=80 - точность предсказаний 78,53%.
- Логистическая регрессия показала наименьшую точность предсказаний из всех обученных моделей - 75,58%.

При тестировании обученных моделей на тестовой выборке результат точности предсказаний оказался следующим: 
- случайный лес (79,47%);
- дерево решений (77,91%);
- логистическая выборка (73,87%)
При этом скорость выполнения предсказаний у моделей дерева решений и логистической регрессии лучше, чем у модели случайного леса. 

Так же, стоит отметить, что обученные модели показали более высокую точность предсказаний, по сравнению с моделью DummyClassifier, в которой все предсказания - это самый часты целевой признак - тариф смарт(0). Точность моделей дерева решений и случайного леса оказалась выше на 9-10%.

Таким образом,  с учетом демонстрируемой точности и скорости выполнения предсказаний рекомендуется модель дерева решений с гипермараметрами: random_state=12345, max_depth=3 и точностью предсказаний 78,53%